<a href="https://colab.research.google.com/github/Dev-180Memes/Summarization-of-Text-and-Docs/blob/main/Summarization_of_Texts_and_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
organizations_cornell_university_arxiv_path = kagglehub.dataset_download('organizations/Cornell-University/arxiv')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json


In [ ]:
!pip install numpy pandas nltk spacy transformers datasets rouge-score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ff06602f993a6c737aaaa89107f164126f1f4c4f933dc6648809e6c0642e6568
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import pandas as pd

# Load the JSON file
data = pd.read_json('/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', lines=True)

# Extract relevant columns (e.g., abstract as input, title as pseudo-summary for evaluation)
texts = data['abstract'].dropna().tolist()  # Input texts
references = data['title'].dropna().tolist()  # Reference summaries (for evaluation)

In [ ]:
import re
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

texts = [preprocess_text(t) for t in texts]
references = [preprocess_text(r) for r in references]

## Extractive Summarization

In [ ]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.5 MB/s eta 0:00:0000:0100:01
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=69b3fe9ee8013dbd9fc7a66309929d0c9b1f944ba5123934e505a78056f6cd13
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e91c3b08bcc72a8ed7345f499cb70342790c45e6e5d6742fb67f064ff094ec26
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
import sumy
print(sumy.__version__)

0.11.0


In [ ]:
from sumy.parsers.plaintext import PlaintextParser

In [ ]:
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [ ]:
def extractive_summary(text, sentence_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return " ".join([str(sentence) for sentence in summary])

In [ ]:
sample_text = texts[0]
extractive_result = extractive_summary(sample_text)
print("Extractive Summary:", extractive_result)

Extractive Summary: good agreement is demonstrated with data from the fermilab tevatron, and predictions are made for more detailed tests with cdf and do data. predictions are shown for distributions of diphoton pairs produced at the energy of the large hadron collider (lhc). distributions of the diphoton pairs from the decay of a higgs boson are contrasted with those produced from qcd processes at the lhc, showing that enhanced sensitivity to the signal can be obtained with judicious selection of events.


## Abstractive Summarization

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def abstractive_summary(text, max_length=100, min_length=30):
    inputs = tokenizer([text], max_length=1024, return_tensors="pt", truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
abstractive_result = abstractive_summary(sample_text)
print("Abstractive Summary:", abstractive_result)

Abstractive Summary: a fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. good agreement is demonstrated with data from the fermilab tevatron, and predictions are made for more detailed tests with cdf and do data.


## Hybrid Approach

In [ ]:
def hybrid_summary(text, sentence_count=3, max_length=100):
    # Step 1: Extract key sentences
    extractive_text = extractive_summary(text, sentence_count)
    # Step 2: Refine with abstractive model
    hybrid_result = abstractive_summary(extractive_text, max_length=max_length)
    return hybrid_result

In [ ]:
hybrid_result = hybrid_summary(sample_text)
print("Hybrid Summary:", hybrid_result)

Hybrid Summary: good agreement is demonstrated with data from the fermilab tevatron. predictions are shown for distributions of diphoton pairs produced at the energy of the large hadron collider.


## Evaluation

### Rouge Score

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
def evaluate_rouge(generated, reference):
    scores = scorer.score(reference, generated)
    return scores

In [ ]:
ref = references[0]
rouge_scores_extractive = evaluate_rouge(extractive_result, ref)
rouge_scores_abstractive = evaluate_rouge(abstractive_result, ref)
rouge_scores_hybrid = evaluate_rouge(hybrid_result, ref)

In [ ]:
print("ROUGE Extractive:", rouge_scores_extractive)
print("ROUGE Abstractive:", rouge_scores_abstractive)
print("ROUGE Hybrid:", rouge_scores_hybrid)

ROUGE Extractive: {'rouge1': Score(precision=0.08860759493670886, recall=0.5833333333333334, fmeasure=0.15384615384615383), 'rouge2': Score(precision=0.01282051282051282, recall=0.09090909090909091, fmeasure=0.02247191011235955), 'rougeL': Score(precision=0.05063291139240506, recall=0.3333333333333333, fmeasure=0.0879120879120879)}
ROUGE Abstractive: {'rouge1': Score(precision=0.13953488372093023, recall=0.5, fmeasure=0.21818181818181817), 'rouge2': Score(precision=0.023809523809523808, recall=0.09090909090909091, fmeasure=0.03773584905660377), 'rougeL': Score(precision=0.11627906976744186, recall=0.4166666666666667, fmeasure=0.1818181818181818)}
ROUGE Hybrid: {'rouge1': Score(precision=0.18518518518518517, recall=0.4166666666666667, fmeasure=0.2564102564102564), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.14814814814814814, recall=0.3333333333333333, fmeasure=0.20512820512820512)}


### BLEU

In [ ]:
import sacrebleu

def evaluate_bleu(generated, reference):
    bleu = sacrebleu.corpus_bleu([generated], [[reference]])
    return bleu.score

In [ ]:
bleu_extractive = evaluate_bleu(extractive_result, ref)
bleu_abstractive = evaluate_bleu(abstractive_result, ref)
bleu_hybrid = evaluate_bleu(hybrid_result, ref)

In [ ]:
print("BLEU Extractive:", bleu_extractive)
print("BLEU Abstractive:", bleu_abstractive)
print("BLEU Hybrid:", bleu_hybrid)

BLEU Extractive: 0.6548987028849833
BLEU Abstractive: 1.2438555540747176
BLEU Hybrid: 1.8196871111910016


In [ ]:
results = {"extractive": [], "abstractive": [], "hybrid": []}
for i in range(min(100, len(texts))):
    text, ref = texts[i], references[i]
    results["extractive"].append(extractive_summary(text))
    results["abstractive"].append(abstractive_summary(text))
    results["hybrid"].append(hybrid_summary(text))

In [ ]:
def avg_scores(results, refs, metric="rouge"):
    scores = []
    for gen, ref in zip(results, refs[:len(results)]):
        if metric == "rouge":
            score = evaluate_rouge(gen, ref)["rougeL"].fmeasure  # Using ROUGE-L F1
        else:
            score = evaluate_bleu(gen, ref)
        scores.append(score)
    return sum(scores) / len(scores)

In [ ]:
for method in results:
    avg_rouge = avg_scores(results[method], references, "rouge")
    avg_bleu = avg_scores(results[method], references, "bleu")
    print(f"{method.capitalize()} - Avg ROUGE-L: {avg_rouge:.4f}, Avg BLEU: {avg_bleu:.4f}")